In [4]:
from openai import OpenAI
import dotenv   
import os

dotenv.load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
# Вариант с доменом без порта (HTTPS):
BASE_URL = "https://llm.t1v.scibox.tech/v1"
# Альтернатива с IP:порт
# BASE_URL = "http://45.145.191.148:4000/v1"

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)


In [5]:

resp = client.chat.completions.create(
    model="qwen3-32b-awq",
    messages=[
        {"role": "system", "content": "Ты дружелюбный помощник"},
        {"role": "user", "content": "Расскажи анекдот"},
    ],
    temperature=0.7,
    top_p=0.9,
    max_tokens=256,
)

print(resp.choices[0].message.content)

<think>
Хорошо, пользователь попросил анекдот. Нужно выбрать подходящий, чтобы он был смешным и не задевал никого. Проверю, нет ли в базе данных анекдотов с оскорбительным содержанием или связанных с неприятными темами. Лучше подойдет что-то простое и понятное, с юмором на бытовую тему.

Может, анекдот про животных или бытовую ситуацию. Например, про кошек или собак. Или про что-то, что происходит в повседневной жизни. Важно, чтобы он был коротким и легко запоминающимся. Проверю, нет ли повторений или клише. Хорошо, вот один: "Идёт мужик по лесу, видит — медведь. Медведь на него. Мужик: «Ты чего?» Медведь: «А ты кто?»". Классический, но работает. Или другой вариант: "Приходит пьяный мужик домой,


In [ ]:
with client.chat.completions.stream(
    model="qwen3-32b-awq",
    messages=[{"role": "user", "content": "Сделай краткое резюме книги Война и мир"}],
    max_tokens=400,
) as stream:
    for event in stream:
        if event.type == "chunk":
            delta = getattr(event.chunk.choices[0].delta, "content", None)
            if delta:
                print(delta, end="", flush=True)
        elif event.type == "message.completed":
            print()  # newlinefrom openai import OpenAI
import dotenv   
import os

dotenv.load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
# Вариант с доменом без порта (HTTPS):
BASE_URL = "https://llm.t1v.scibox.tech/v1"
# Альтернатива с IP:порт
# BASE_URL = "http://45.145.191.148:4000/v1"

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)


<think>
Хорошо, мне нужно сделать краткое резюме книги "Война и мир" Льва Толстого. Начну с того, что вспомню основные сюжетные линии и главных героев. Важно упомянуть основные темы, такие как война, мир, любовь и судьба. Надо не забыть про исторический контекст — Отечественная война 1812 года. Возможно, стоит структурировать резюме по частям или ключевым эпизодам, но так как нужно краткое, то лучше обобщить.

Главные семьи: Ростовы, Болконские, Курагины. Нужно описать их судьбы. Например, Наташа Ростова и ее отношения с Андреем Болконским, потом с Николаем Ростовым. Также важна личность Пьера Безухова, его поиск смысла жизни и участие в войне. Не забыть про исторические персонажи, таких как Кутузов, Наполеон.

Темы: противостояние войны и мирной жизни, роль личности в истории, поиск смысла жизни. Толстой критикует романтизацию войны, показывает человеческие страдания. Также важно упомянуть эпиграф — мысль о том, что люди не делают историю, а история делает людей.

Нужно проверить, что